# Examples

In [1]:
from dataclasses import dataclass
from datetime import datetime, timedelta
from copy import copy
from xardano import Wallet, XardanoLedger, TokenValue, add_tokens, CNA, xadas, zights, MIN_XADA, ONE_XADA, ONE_ZIGHT, debug, ensure, AbstractContract, ValidationException

debugging enabled


## Simple Transfers

In [2]:
ledger = XardanoLedger()
alice = Wallet(ledger)
bob = Wallet(ledger)

ledger.send_faucet_tokens(alice, xadas(10000))
print(alice.balance())

alice.send({bob.pk : {CNA.XLOV : 10 * ONE_XADA}})
print(bob.balance())
bob.send({alice.pk : {CNA.XLOV : .1 * ONE_XADA}})


AbstractUtxo b'n\xee\r6Z-\xf0Y\x01v \x8fd\xaf\xd5\x96c\x87~\x10\xc7\xc2f4\xec\x8b\x9f\xb2No\xf6\x91'
{<CNA.XLOV: 0>: 10000000000}
AbstractUtxo b'r&\xdae\x94Q\xde\x99\xfc\x99JK\xb2=c\x81\x83G\xbd7.\x0c\x1aQ\xd8r\xcb\xfb\xf6m\xf9G'
AbstractUtxo b'\xfaE\xd0%\xb0\xbf\xfa\x01C\xee\xecH\x84\xd6\xea\x97\x89,\x10\xbf\x10\x18\x8f\xd8\xc2%\xf2*G-\xb4\xe9'
self.outputs {PlainUtxo(pk:b'U4PXKKMl', {<CNA.XLOV: 0>: 9989000000}), PlainUtxo(pk:b'mSpDwrAk', {<CNA.XLOV: 0>: 10000000})}
hash(([b'n\xee\r6Z-\xf0Y\x01v \x8fd\xaf\xd5\x96c\x87~\x10\xc7\xc2f4\xec\x8b\x9f\xb2No\xf6\x91'], [PlainUtxo(pk:b'U4PXKKMl', {<CNA.XLOV: 0>: 9989000000}), PlainUtxo(pk:b'mSpDwrAk', {<CNA.XLOV: 0>: 10000000})]))
tx_hash bytearray(b"([b\'n\\xee\\r6Z-\\xf0Y\\x01v \\x8fd\\xaf\\xd5\\x96c\\x87~\\x10\\xc7\\xc2f4\\xec\\x8b\\x9f\\xb2No\\xf6\\x91\'], [PlainUtxo(pk:b\'U4PXKKMl\', {<CNA.XLOV: 0>: 9989000000}), PlainUtxo(pk:b\'mSpDwrAk\', {<CNA.XLOV: 0>: 10000000})])")
extra_inputs []
self.inputs {b'n\xee\r6Z-\xf0Y\x01v \x8fd\xaf\xd5\x9

In [3]:
alice.balance(), bob.balance()

({<CNA.XLOV: 0>: 9989100000.0}, {<CNA.XLOV: 0>: 8900000.0})

## Contracts

### PushPop Example

In [4]:
class PushPop(AbstractContract):
  def __init__(self) -> None:
    super().__init__()
    self.stack = []
    self.limit = 1
  def push(self, x, tithe: TokenValue):
    debug(f'**** PUSH {x}')
    ensure(len(self.stack) < self.limit, "stack overflow")
    price = 2 * self.limit * ONE_XADA
    #match tithe:
    #  case TokenValue(CNA.XLOV, price):
    #    self.stack.append(x)
    #  case _:
    #    raise InsufficientFunds()
    self.stack.append(x)
    return len(self.stack)
  def pop(self, pk):
    debug(f'**** POP')
    ensure(len(self.stack) > 0, "stack underflow")
    x = self.stack.pop()
    self.ctx.send(pk, {CNA.XLOV : self.limit * ONE_XADA})
    return x

In [5]:
101000000 / ONE_XADA, alice.balance()[CNA.XLOV] / ONE_XADA

(101.0, 9989.1)

In [6]:
ledger = XardanoLedger()
alice = Wallet(ledger)
# faucet funding
ledger.send_faucet_tokens(alice, xadas(10000))

# deploy the pp contract
pp = alice.deploy(PushPop(), {CNA.XLOV : MIN_XADA})
ledger.utxos_by_hash[pp]

AbstractUtxo b'xA\x1a\xed\xa8\x91\x88\x92\xc1c\xa7\x956\n\xeb\xe9\xa1\x9fl\x89\x0c\x9dw\xa0\xc0\xd1\xae\x01\xb6\xd7\xd1X'
deploy <class '__main__.PushPop'> with toks {<CNA.XLOV: 0>: 1000000}
needed = {<CNA.XLOV: 0>: 101000000}
AbstractUtxo b'\xc2Xw\x1b6Kcl\xacz&\xcf*9Y!\xa6-3.\xe2\x9fJ\xdc\x9aPc\xc6\xcc\x8eH2'
self.outputs {PlainUtxo(pk:b'ofFvAIuv', {<CNA.XLOV: 0>: 9899000000})}
hash(([b'xA\x1a\xed\xa8\x91\x88\x92\xc1c\xa7\x956\n\xeb\xe9\xa1\x9fl\x89\x0c\x9dw\xa0\xc0\xd1\xae\x01\xb6\xd7\xd1X'], [PlainUtxo(pk:b'ofFvAIuv', {<CNA.XLOV: 0>: 9899000000})]))
tx_hash bytearray(b"([b\'xA\\x1a\\xed\\xa8\\x91\\x88\\x92\\xc1c\\xa7\\x956\\n\\xeb\\xe9\\xa1\\x9fl\\x89\\x0c\\x9dw\\xa0\\xc0\\xd1\\xae\\x01\\xb6\\xd7\\xd1X\'], [PlainUtxo(pk:b\'ofFvAIuv\', {<CNA.XLOV: 0>: 9899000000})])")
extra_inputs []
self.inputs {b'xA\x1a\xed\xa8\x91\x88\x92\xc1c\xa7\x956\n\xeb\xe9\xa1\x9fl\x89\x0c\x9dw\xa0\xc0\xd1\xae\x01\xb6\xd7\xd1X'}
IN utxos to sum [PlainUtxo(pk:b'ofFvAIuv', {<CNA.XLOV: 0>: 10000000000})]
utxo t

ContractUtxo(<class '__main__.PushPop'>, {<CNA.XLOV: 0>: 1000000})

In [7]:
# call the pp contract
alice.call(pp, 'push', 'a', TokenValue(CNA.XLOV, 33 * ONE_XADA))
ledger.utxos_by_hash[pp]

arg a
arg TokenValue(cna=<CNA.XLOV: 0>, num=33000000)
AbstractUtxo b'\xb3u\xea\xb3t\xe9t\x8dujw@G\xc8)m\xda\x15\xa4\xd3W*l><OR\x8e\xc8\xc65\x9f'
self.outputs {PlainUtxo(pk:b'ofFvAIuv', {<CNA.XLOV: 0>: 9856000000})}
hash(([b'\xc2Xw\x1b6Kcl\xacz&\xcf*9Y!\xa6-3.\xe2\x9fJ\xdc\x9aPc\xc6\xcc\x8eH2'], [PlainUtxo(pk:b'ofFvAIuv', {<CNA.XLOV: 0>: 9856000000})]))
tx_hash bytearray(b"([b\'\\xc2Xw\\x1b6Kcl\\xacz&\\xcf*9Y!\\xa6-3.\\xe2\\x9fJ\\xdc\\x9aPc\\xc6\\xcc\\x8eH2\'], [PlainUtxo(pk:b\'ofFvAIuv\', {<CNA.XLOV: 0>: 9856000000})])")
HERE-------------------------------------------------------------
arg a
arg TokenValue(cna=<CNA.XLOV: 0>, num=33000000)
harvested = {<CNA.XLOV: 0>: 33000000}
**** PUSH a
=========== push(...) => 1
all contracts called [ContractUtxo(<class '__main__.PushPop'>, {<CNA.XLOV: 0>: 34000000})]
AbstractUtxo b"\xd4[*a%w\xc0\xf7@\x19Y\xdb\xdcC\x93\xf2\x8a'\xc3\x9c\xfa`A\x14I=\xd9tm\x13\xab\xee"
extra_inputs [ContractUtxo(<class '__main__.PushPop'>, {<CNA.XLOV: 0>: 1000000})]
sel

ContractUtxo(<class '__main__.PushPop'>, {<CNA.XLOV: 0>: 34000000})

In [8]:
try:
  alice.call(pp, 'push', 'a', TokenValue(CNA.XLOV, 2 * ONE_XADA))
  raise Exception('second push should fail')
except ValidationException:
  pass

arg a
arg TokenValue(cna=<CNA.XLOV: 0>, num=2000000)
AbstractUtxo b'\xbd\x84r\xa2O;]\xda\xee\xae\x90m~Z$%\xc1\x82W!J\xe3\xb2\xf8\xc0~\xb4C6\x12>}'
self.outputs {PlainUtxo(pk:b'ofFvAIuv', {<CNA.XLOV: 0>: 9844000000})}
hash(([b'\xb3u\xea\xb3t\xe9t\x8dujw@G\xc8)m\xda\x15\xa4\xd3W*l><OR\x8e\xc8\xc65\x9f'], [PlainUtxo(pk:b'ofFvAIuv', {<CNA.XLOV: 0>: 9844000000})]))
tx_hash bytearray(b"([b\'\\xb3u\\xea\\xb3t\\xe9t\\x8dujw@G\\xc8)m\\xda\\x15\\xa4\\xd3W*l><OR\\x8e\\xc8\\xc65\\x9f\'], [PlainUtxo(pk:b\'ofFvAIuv\', {<CNA.XLOV: 0>: 9844000000})])")
HERE-------------------------------------------------------------
arg a
arg TokenValue(cna=<CNA.XLOV: 0>, num=2000000)
harvested = {<CNA.XLOV: 0>: 2000000}
**** PUSH a
exception in call: stack overflow
=========== !!! push(...) => stack overflow


In [9]:
alice.call(pp, 'pop', alice.pk)

arg pk:b'ofFvAIuv'
AbstractUtxo b'a\xcc\x9c,V\xf4\\x\x8a\x98\xaf\x9cl\x10R?\xfd!w%q#\r\xfa\x1e\x86p\xee \xe8\xed\x0e'
self.outputs {PlainUtxo(pk:b'ofFvAIuv', {<CNA.XLOV: 0>: 9846000000})}
hash(([b'\xb3u\xea\xb3t\xe9t\x8dujw@G\xc8)m\xda\x15\xa4\xd3W*l><OR\x8e\xc8\xc65\x9f'], [PlainUtxo(pk:b'ofFvAIuv', {<CNA.XLOV: 0>: 9846000000})]))
tx_hash bytearray(b"([b\'\\xb3u\\xea\\xb3t\\xe9t\\x8dujw@G\\xc8)m\\xda\\x15\\xa4\\xd3W*l><OR\\x8e\\xc8\\xc65\\x9f\'], [PlainUtxo(pk:b\'ofFvAIuv\', {<CNA.XLOV: 0>: 9846000000})])")
HERE-------------------------------------------------------------
arg pk:b'ofFvAIuv'
harvested = {}
**** POP
attempt to send {<CNA.XLOV: 0>: 1000000} from {<CNA.XLOV: 0>: 34000000}
my bag {<CNA.XLOV: 0>: 34000000}
AbstractUtxo b'\xca2\xfd\x93NB\xf1\xe4\x81\x10[\x8d\xce\x1c\xe9\xe6\x02\xd62\x0e\xd6*\xd5\xff\xb1\x12\xe1\x02\x82\x8c\xa7\xf5'
=========== pop(...) => a
all contracts called [ContractUtxo(<class '__main__.PushPop'>, {<CNA.XLOV: 0>: 33000000})]
AbstractUtxo b'9\x05\x899\xc

'a'

In [10]:
try:
  alice.call(pp, 'pop', alice.pk)
  raise Exception('second push should raise stack underflow')
except ValidationException:
  pass

arg pk:b'ofFvAIuv'
AbstractUtxo b'\xb4\xb2Hi\x11|\xfa\x193V\x1eH\xf1\xa6)\xe0Xn$\xb4Nx"(\xcf)\xc8\x12$\x01.\x91'
self.outputs {PlainUtxo(pk:b'ofFvAIuv', {<CNA.XLOV: 0>: 9836000000})}
hash(([b'a\xcc\x9c,V\xf4\\x\x8a\x98\xaf\x9cl\x10R?\xfd!w%q#\r\xfa\x1e\x86p\xee \xe8\xed\x0e'], [PlainUtxo(pk:b'ofFvAIuv', {<CNA.XLOV: 0>: 9836000000})]))
tx_hash bytearray(b"([b\'a\\xcc\\x9c,V\\xf4\\\\x\\x8a\\x98\\xaf\\x9cl\\x10R?\\xfd!w%q#\\r\\xfa\\x1e\\x86p\\xee \\xe8\\xed\\x0e\'], [PlainUtxo(pk:b\'ofFvAIuv\', {<CNA.XLOV: 0>: 9836000000})])")
HERE-------------------------------------------------------------
arg pk:b'ofFvAIuv'
harvested = {}
**** POP
exception in call: stack underflow
=========== !!! pop(...) => stack underflow


### Token Allocation Example

In [11]:
@dataclass
class ToyAlloc:
  dest_addr: str
  num_ztars: int
  thaw_dttm: datetime

class ToyRedemption(AbstractContract):
  def __init__(self, alloc: ToyAlloc) -> None:
    super().__init__()
    self.alloc = alloc
  def release(self):
    debug(f'**** RELEASE {self.alloc.dest_addr}')
    ensure(self.ctx.ddtm >= self.alloc.thaw_dttm, 'not yet thawed')
    toks = {CNA.ZTAR : self.alloc.num_ztars}
    add_tokens(toks, CNA.XLOV, MIN_XADA) # refund
    debug(f'toks to redeemer {toks}')
    self.ctx.send(self.alloc.dest_addr, toks)
    TokenValue(CNA.ZTAR, self.ctx.halt(self.alloc.num_ztars))

class ToyShard(AbstractContract):
  def __init__(self, allocations: dict) -> None:
    super().__init__()
    self.allocations = copy(allocations)
    self.claimed = dict()
  def claim(self, key, claim_deposit):
    debug(f'**** CLAIM {key} allocations {self.allocations} claimed {self.claimed}')
    ensure(key in self.allocations, 'allocation does not exist')
    ensure(key not in self.claimed, 'allocation already claimed')
    debug(f'claim_deposit {claim_deposit}')
    ensure(claim_deposit == xadas(1), 'claim must include deposit for redemption utxo')
    alloc = self.allocations[key]
    debug(f'alloc {alloc}')
    self.claimed[key] = True
    toks = {CNA.ZTAR : alloc.num_ztars}
    add_tokens(toks, CNA.XLOV, MIN_XADA) # include a min_xada for the redemption
    rdm = self.ctx.deploy(ToyRedemption(alloc), toks)
    debug(f'rdm {rdm}')
    return rdm

In [12]:
ledger = XardanoLedger()
tge = Wallet(ledger)
alice = Wallet(ledger)
bob = Wallet(ledger)
charlie = Wallet(ledger)
# faucet funding
for w in [tge, alice, bob, charlie]:
  ledger.send_faucet_tokens(w, xadas(10000))
# give the TGE some ZIGHT tokens to distribute
ledger.send_faucet_tokens(tge, zights(10000))
# Invent some allocations
allocs = {
    id(alice)   : ToyAlloc(alice.pk,     10, ledger.time + timedelta(days=10)),
    id(bob)     : ToyAlloc(bob.pk,      100, ledger.time + timedelta(days=11)),
    id(charlie) : ToyAlloc(charlie.pk, 1000, ledger.time + timedelta(days=12)),
}
# deploy the shard contract
min_xadas = 1 # use the following to fund all claims: len(allocs) + 1
shard = tge.deploy(ToyShard(allocs), {CNA.XLOV : min_xadas * MIN_XADA, CNA.ZTAR : 1111})
ledger.utxos_by_hash[shard]

AbstractUtxo b'hhf\x80cUl\x90v{5\xf1\x01Z\x1c\xb7\x17\xd9\x8d\x19\xee\x02%!<\x9c\r8\xf7\xa0\x84B'
AbstractUtxo b'\xdf;\xf74\xe1\xf4\xffe\x07\x13%\xc12\xf7\xc7.\xd4\xf9\xff/\xe8C\xc0\x19\x0f\x80\xe8\x01\xa9\xc8\x1b\xa9'
AbstractUtxo b')\x81\xb1\x1e\xa4|\x08D&\xbcV?/\xf6\x05\xd6\xb5"Y\xc2}R\xc0\x02k&8\xcc?\xcf\x9e]'
AbstractUtxo b'\x8d\x18\xff\xdd\x8a7\x03\x1fY\xbd\xa4i\x9b\xc5\xfd\xc5\xf8\xcdU\xebzq\x9e^\xca(\xb5@\xe8\xb6\x95\xcb'
AbstractUtxo b')\r\x8e\xd4\x8fLMR\xf6[*f^T\xd4\x08\xa8\xe9fX\xe6N\xd0\xbcR\xd8\xbb-\xa3\xb9U\x1e'
deploy <class '__main__.ToyShard'> with toks {<CNA.XLOV: 0>: 1000000, <CNA.ZTAR: 1>: 1111}
needed = {<CNA.XLOV: 0>: 101000000, <CNA.ZTAR: 1>: 1111}
AbstractUtxo b"\xc5!\xd5\x97\xd3\xd8\xfc\xbd3\x12\xb3\x1b'Q\xca\xad\x86\x90R\x96;\x9c\t\xb5R\xf9da\x1d\x9c\x88l"
self.outputs {PlainUtxo(pk:b'/fONfrOR', {<CNA.XLOV: 0>: 9899000000, <CNA.ZTAR: 1>: 9999998889})}
hash(([b'hhf\x80cUl\x90v{5\xf1\x01Z\x1c\xb7\x17\xd9\x8d\x19\xee\x02%!<\x9c\r8\xf7\xa0\x84B', b')\r\x8e\xd4\x8f

ContractUtxo(<class '__main__.ToyShard'>, {<CNA.XLOV: 0>: 1000000, <CNA.ZTAR: 1>: 1111})

In [13]:
before = alice.balance()
alice_rdm = alice.call(shard, 'claim', id(alice), xadas(1))
before, alice.balance()

arg 4406556480
arg TokenValue(cna=<CNA.XLOV: 0>, num=1000000)
AbstractUtxo b'R\xd0\xc4F!1\x8f\x1d\xe9m\xe8EZG\xc9h\xca\xab#BhV\xabj\xfbY_;uN\xf1a'
self.outputs {PlainUtxo(pk:b'hCHz+VCd', {<CNA.XLOV: 0>: 9989000000})}
hash(([b'\xdf;\xf74\xe1\xf4\xffe\x07\x13%\xc12\xf7\xc7.\xd4\xf9\xff/\xe8C\xc0\x19\x0f\x80\xe8\x01\xa9\xc8\x1b\xa9'], [PlainUtxo(pk:b'hCHz+VCd', {<CNA.XLOV: 0>: 9989000000})]))
tx_hash bytearray(b"([b\'\\xdf;\\xf74\\xe1\\xf4\\xffe\\x07\\x13%\\xc12\\xf7\\xc7.\\xd4\\xf9\\xff/\\xe8C\\xc0\\x19\\x0f\\x80\\xe8\\x01\\xa9\\xc8\\x1b\\xa9\'], [PlainUtxo(pk:b\'hCHz+VCd\', {<CNA.XLOV: 0>: 9989000000})])")
HERE-------------------------------------------------------------
arg 4406556480
arg TokenValue(cna=<CNA.XLOV: 0>, num=1000000)
harvested = {<CNA.XLOV: 0>: 1000000}
**** CLAIM 4406556480 allocations {4406556480: ToyAlloc(dest_addr=pk:b'hCHz+VCd', num_ztars=10, thaw_dttm=datetime.datetime(2024, 8, 6, 12, 19, 59, 563989)), 4406557200: ToyAlloc(dest_addr=pk:b'3z5zpi6Q', num_ztars=100, th

({<CNA.XLOV: 0>: 10000000000}, {<CNA.XLOV: 0>: 9989000000})

Alice cannot release funds from her redemption contract until the thaw period expires.

In [14]:
try:
  alice.balance(), alice.call(alice_rdm, 'release'), alice.balance()
except Exception as ex:
  print(ex)

AbstractUtxo b'zz\xc8\xed\xa5y\x9b\xf5h\xd6\xff\xee\x0bq\xeeb\xfbN\x00)\xc4\x91u\xdegK\xa4\xa7\xff\x92\xb0e'
self.outputs {PlainUtxo(pk:b'hCHz+VCd', {<CNA.XLOV: 0>: 9979000000})}
hash(([b'R\xd0\xc4F!1\x8f\x1d\xe9m\xe8EZG\xc9h\xca\xab#BhV\xabj\xfbY_;uN\xf1a'], [PlainUtxo(pk:b'hCHz+VCd', {<CNA.XLOV: 0>: 9979000000})]))
tx_hash bytearray(b"([b\'R\\xd0\\xc4F!1\\x8f\\x1d\\xe9m\\xe8EZG\\xc9h\\xca\\xab#BhV\\xabj\\xfbY_;uN\\xf1a\'], [PlainUtxo(pk:b\'hCHz+VCd\', {<CNA.XLOV: 0>: 9979000000})])")
HERE-------------------------------------------------------------
harvested = {}
**** RELEASE pk:b'hCHz+VCd'
exception in call: not yet thawed
=========== !!! release(...) => not yet thawed
not yet thawed


In [15]:
ledger.sleep(timedelta(days=10))
alice.balance(), alice.call(alice_rdm, 'release'), alice.balance()

AbstractUtxo b"\x17\x82\xdb\xe8\xbf\xf5b\x1cfO\x16-\xd6\x82\x80.\xadiL'\x8d2\x84b\xaf\x8fv\\\xd298L"
self.outputs {PlainUtxo(pk:b'hCHz+VCd', {<CNA.XLOV: 0>: 9979000000})}
hash(([b'R\xd0\xc4F!1\x8f\x1d\xe9m\xe8EZG\xc9h\xca\xab#BhV\xabj\xfbY_;uN\xf1a'], [PlainUtxo(pk:b'hCHz+VCd', {<CNA.XLOV: 0>: 9979000000})]))
tx_hash bytearray(b"([b\'R\\xd0\\xc4F!1\\x8f\\x1d\\xe9m\\xe8EZG\\xc9h\\xca\\xab#BhV\\xabj\\xfbY_;uN\\xf1a\'], [PlainUtxo(pk:b\'hCHz+VCd\', {<CNA.XLOV: 0>: 9979000000})])")
HERE-------------------------------------------------------------
harvested = {}
**** RELEASE pk:b'hCHz+VCd'
toks to redeemer {<CNA.ZTAR: 1>: 10, <CNA.XLOV: 0>: 1000000}
attempt to send {<CNA.ZTAR: 1>: 10, <CNA.XLOV: 0>: 1000000} from {<CNA.ZTAR: 1>: 10, <CNA.XLOV: 0>: 1000000}
my bag {<CNA.ZTAR: 1>: 10, <CNA.XLOV: 0>: 1000000}
AbstractUtxo b'\x89\x89\x9eDl\x1b\xd9gR\x00\x03\xcfp\xec\xa7s0:/\xe2;\t\xc6! \x1e\x96#\x0c\x8aA&'
halt ContractUtxo(<class '__main__.ToyRedemption'>, {<CNA.ZTAR: 1>: 0, <CNA.XLOV: 0>: 0})

({<CNA.XLOV: 0>: 9989000000}, 10, {<CNA.XLOV: 0>: 9979000000})

Now the redemption has been released, it no longer exists.

In [16]:
try:
  alice.balance(), alice.call(alice_rdm, 'release'), alice.balance()
except Exception as ex:
  print(ex)

AbstractUtxo b'b\xa0\n\xa2\x97\xa4\x9e\x9bL\'&"xe\xdaO\xd6\x91\xbb\t\x9c\x06\xfdT\xc5\x03\xb7+\xff\x15\x15\xb4'
self.outputs {PlainUtxo(pk:b'hCHz+VCd', {<CNA.XLOV: 0>: 9969000000})}
hash(([b"\x17\x82\xdb\xe8\xbf\xf5b\x1cfO\x16-\xd6\x82\x80.\xadiL'\x8d2\x84b\xaf\x8fv\\\xd298L"], [PlainUtxo(pk:b'hCHz+VCd', {<CNA.XLOV: 0>: 9969000000})]))
tx_hash bytearray(b'([b"\\x17\\x82\\xdb\\xe8\\xbf\\xf5b\\x1cfO\\x16-\\xd6\\x82\\x80.\\xadiL\'\\x8d2\\x84b\\xaf\\x8fv\\\\\\xd298L"], [PlainUtxo(pk:b\'hCHz+VCd\', {<CNA.XLOV: 0>: 9969000000})])')
contract not found b'\xf7\x17\xe4\x17\xc2\xddq\xb9\x9b)=#\x14O\xb5\xef\x9f\xfb\xbal6)\x19^\xd1\x92\xc6{\x82TQ0'


And Alice cannot make another claim.

In [17]:
try:
  alice_rdm = alice.call(shard, 'claim', id(alice), xadas(1))
except Exception as ex:
  print(ex)

arg 4406556480
arg TokenValue(cna=<CNA.XLOV: 0>, num=1000000)
AbstractUtxo b'\xa8\xdbU\xf8\xb1)\x0e\r\xb3Sy\x90!k\x9d:#d\xf5\x10\xebQ\xed\xe4\xba\xe8\xc9;\xa9\x99\xe4\xbf'
self.outputs {PlainUtxo(pk:b'hCHz+VCd', {<CNA.XLOV: 0>: 9968000000})}
hash(([b"\x17\x82\xdb\xe8\xbf\xf5b\x1cfO\x16-\xd6\x82\x80.\xadiL'\x8d2\x84b\xaf\x8fv\\\xd298L"], [PlainUtxo(pk:b'hCHz+VCd', {<CNA.XLOV: 0>: 9968000000})]))
tx_hash bytearray(b'([b"\\x17\\x82\\xdb\\xe8\\xbf\\xf5b\\x1cfO\\x16-\\xd6\\x82\\x80.\\xadiL\'\\x8d2\\x84b\\xaf\\x8fv\\\\\\xd298L"], [PlainUtxo(pk:b\'hCHz+VCd\', {<CNA.XLOV: 0>: 9968000000})])')
HERE-------------------------------------------------------------
arg 4406556480
arg TokenValue(cna=<CNA.XLOV: 0>, num=1000000)
harvested = {<CNA.XLOV: 0>: 1000000}
**** CLAIM 4406556480 allocations {4406556480: ToyAlloc(dest_addr=pk:b'hCHz+VCd', num_ztars=10, thaw_dttm=datetime.datetime(2024, 8, 6, 12, 19, 59, 563989)), 4406557200: ToyAlloc(dest_addr=pk:b'3z5zpi6Q', num_ztars=100, thaw_dttm=datetime.date

But the shard is still there, and Bob can still claim.

In [18]:
before = bob.balance()
bob_rdm = bob.call(shard, 'claim', id(bob), xadas(1))
before, bob.balance()

arg 4406557200
arg TokenValue(cna=<CNA.XLOV: 0>, num=1000000)
AbstractUtxo b'\xc9\xcf\xda\xe2\xc7B\xa7\x85\xbc\x1f\x9d\xaf2"\xcf\x86?:_\xddu\xcb\xc6\xb1eJ\x92\x8c\xa1\x9e\xbf.'
self.outputs {PlainUtxo(pk:b'3z5zpi6Q', {<CNA.XLOV: 0>: 9989000000})}
hash(([b')\x81\xb1\x1e\xa4|\x08D&\xbcV?/\xf6\x05\xd6\xb5"Y\xc2}R\xc0\x02k&8\xcc?\xcf\x9e]'], [PlainUtxo(pk:b'3z5zpi6Q', {<CNA.XLOV: 0>: 9989000000})]))
tx_hash bytearray(b'([b\')\\x81\\xb1\\x1e\\xa4|\\x08D&\\xbcV?/\\xf6\\x05\\xd6\\xb5"Y\\xc2}R\\xc0\\x02k&8\\xcc?\\xcf\\x9e]\'], [PlainUtxo(pk:b\'3z5zpi6Q\', {<CNA.XLOV: 0>: 9989000000})])')
HERE-------------------------------------------------------------
arg 4406557200
arg TokenValue(cna=<CNA.XLOV: 0>, num=1000000)
harvested = {<CNA.XLOV: 0>: 1000000}
**** CLAIM 4406557200 allocations {4406556480: ToyAlloc(dest_addr=pk:b'hCHz+VCd', num_ztars=10, thaw_dttm=datetime.datetime(2024, 8, 6, 12, 19, 59, 563989)), 4406557200: ToyAlloc(dest_addr=pk:b'3z5zpi6Q', num_ztars=100, thaw_dttm=datetime.datetime(

({<CNA.XLOV: 0>: 10000000000}, {<CNA.XLOV: 0>: 9989000000})

And, as currently defined, there is no access control on the redemption contract.  This is "safe" in the sense that it can deposit tokens only to the predetermined destination address, but may have tax implications for the recipient.

In [19]:
ledger.sleep(timedelta(days=1))
alice.balance(), bob.balance(), alice.call(bob_rdm, 'release'), alice.balance(), bob.balance()

AbstractUtxo b'J\x7f\x9e\xcc\x0cf\xf4\xed\x19\xaf\xae\xe64\xb4k\x02)\xc2\x18RN \x04\x7f\xaeI3\xc3B\xd5\x19\x97'
self.outputs {PlainUtxo(pk:b'hCHz+VCd', {<CNA.XLOV: 0>: 9969000000})}
hash(([b"\x17\x82\xdb\xe8\xbf\xf5b\x1cfO\x16-\xd6\x82\x80.\xadiL'\x8d2\x84b\xaf\x8fv\\\xd298L"], [PlainUtxo(pk:b'hCHz+VCd', {<CNA.XLOV: 0>: 9969000000})]))
tx_hash bytearray(b'([b"\\x17\\x82\\xdb\\xe8\\xbf\\xf5b\\x1cfO\\x16-\\xd6\\x82\\x80.\\xadiL\'\\x8d2\\x84b\\xaf\\x8fv\\\\\\xd298L"], [PlainUtxo(pk:b\'hCHz+VCd\', {<CNA.XLOV: 0>: 9969000000})])')
HERE-------------------------------------------------------------
harvested = {}
**** RELEASE pk:b'3z5zpi6Q'
toks to redeemer {<CNA.ZTAR: 1>: 100, <CNA.XLOV: 0>: 1000000}
attempt to send {<CNA.ZTAR: 1>: 100, <CNA.XLOV: 0>: 1000000} from {<CNA.ZTAR: 1>: 100, <CNA.XLOV: 0>: 1000000}
my bag {<CNA.ZTAR: 1>: 100, <CNA.XLOV: 0>: 1000000}
AbstractUtxo b'\xf1%\\H\x18L^[\xc8\x14\xd9\xb1\xb0\x8b\x95.\xd4\xca\x82\xd4.\xa0R\xc7\xd3;\xd2[Xc\xdbe'
halt ContractUtxo(<class '__ma

({<CNA.XLOV: 0>: 9979000000},
 {<CNA.XLOV: 0>: 9989000000},
 100,
 {<CNA.XLOV: 0>: 9969000000},
 {<CNA.XLOV: 0>: 9989000000})